In [13]:
import os
import numpy as np
import pandas as pd
import datetime
from pathlib import Path
from pprint import pprint
import configparser
import ezomero
from ezomero import rois
#   https://thejacksonlaboratory.github.io/ezomero/ezomero.html#module-ezomero
#   https://docs.openmicroscopy.org/ome-model/5.6.3/developers/roi.html
#   https://github.com/HdBraanker/Spectral_Flow_Workflow/blob/main/script.Rmd
datestr = datetime.datetime.now().strftime('%Y_%m_%d')
print(datestr)

2024_07_17


In [14]:
# Settings for OMERO
home = str(Path.home())
config = configparser.RawConfigParser()
config.read(r'{}\Documents\myKeyVariables.cfg'.format(home))   
envVars = dict(config.items('OMERO'))

# Settings for OMERO
user = envVars['user']
password = envVars['pass']
host = envVars['url']
port = 4064 #4080
secure = True #False
outDir = envVars['pointsdownloaded']

conn = ezomero.connect(user, password, 'Nguyen.J_Lab', host=host, port=port, secure=secure)
ezomero.print_projects(conn)

Projects:
	PCF_2024_Transplant_Reaction_in_LN:	1254


In [16]:
allSamps = ezomero.get_dataset_ids(conn, project=1254)
# allSamps.extend( ezomero.get_dataset_ids(conn, project=1254) )

getImageByName = {}
for im in allSamps:
    imgDesc, pix = ezomero.get_image(conn, im, xyzct=False, no_pixels=True)
    imName = imgDesc.getName().replace(".ome.tiff","")
    getImageByName[imName] = im

print("There are {} datasets to investigate. [{}]".format(
    len(allSamps), ', '.join(getImageByName.keys() )))

There are 5 datasets to investigate. [Tonsil21_BMS_region_006, Tonsil21_BMS_region_002, Tonsil21_BMS_region_001, Tonsil21_BMS_region_007, Tonsil21_BMS_region_020]


In [7]:
## Image Desc
## https://docs.openmicroscopy.org/omero/5.4.1/developers/PythonBlitzGateway.html
# https://downloads.openmicroscopy.org/omero/5.4.10/api/python/omero/omero.gateway.html#omero.gateway._ImageWrapper
fout = open(os.path.join(outDir,'log_omero_{}.txt'.format(datestr) ), 'w')
for sm in allSamps:
    directory_path = os.path.join(outDir,"Dataset_"+str(sm))
    # Check if the directory exists
    if not os.path.exists(directory_path):
        # Create the directory
        os.makedirs(directory_path)
        print(f"Directory '{directory_path}' created successfully.")
    
    imgs = ezomero.get_image_ids(conn, dataset=sm)
    for im in imgs:
        imgDesc, pix = ezomero.get_image(conn, im, xyzct=False, no_pixels=True)
        if(imgDesc.getROICount(shapeType="Point") > 0):
            fout.write(imgDesc.getName()+'\n')
            shName = imgDesc.getName().replace(".ome.tiff",'')
            imgPointsIDs = ezomero.get_roi_ids(conn, im)
            print("{} has {}".format(imgDesc.getName().replace('\\.ome\\.tiff',''), len(imgPointsIDs)))
            fout.write("{} Annotations\n".format(len(imgPointsIDs)))
            allThesePoints = []
            for i in imgPointsIDs:
                for ppI in ezomero.get_shape_ids(conn, i):
                    try:
                        pt, fill, border, width = ezomero.get_shape(conn, ppI)
                        allThesePoints.append([pt.x, pt.y, pt.label,pt.label, fill, border, width])
                    except:
                        print("   BAD point ID : "+str(i))
            #pprint(allThesePoints)
            pointTable = pd.DataFrame(allThesePoints)
            pointTable.columns = ["x","y","class", "name","fill","border","width"]
            cntTbl = pointTable['class'].value_counts().to_frame()
            noms = cntTbl.index.to_list()
            cnts = cntTbl.values
            for n in range(len(noms)):
                fout.write("{}\t{}\n".format(noms[n],cnts[n][0]))
            fout.write("\n")
            pointTable.to_csv(os.path.join(directory_path,shName+'-points.tsv'), index=False, sep="\t")


Directory 'Y:\AnalysisPipelineProjects\DataFreeze_BMSA1_ClassificationPaper\OMERO_POINTS\Dataset_2012' created successfully.
Directory 'Y:\AnalysisPipelineProjects\DataFreeze_BMSA1_ClassificationPaper\OMERO_POINTS\Dataset_2013' created successfully.
Directory 'Y:\AnalysisPipelineProjects\DataFreeze_BMSA1_ClassificationPaper\OMERO_POINTS\Dataset_2014' created successfully.
Directory 'Y:\AnalysisPipelineProjects\DataFreeze_BMSA1_ClassificationPaper\OMERO_POINTS\Dataset_2015' created successfully.
Slide1_B3.ome.tif has 4
Slide5_D1.ome.tif has 1
Directory 'Y:\AnalysisPipelineProjects\DataFreeze_BMSA1_ClassificationPaper\OMERO_POINTS\Dataset_2016' created successfully.
Slide5_F3.ome.tif has 858
Slide1_B3.ome.tif has 4
Slide5_D1.ome.tif has 1
Slide5_F3.ome.tif has 858


In [8]:
fout.close()